# Space Missions Analysis - Complete Solution

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Import Statements

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from iso3166 import countries
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [3]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [4]:
# Shape of the data
print(f"Shape: {df_data.shape}")
print(f"Rows: {df_data.shape[0]}")
print(f"Columns: {df_data.shape[1]}")

Shape: (4324, 9)
Rows: 4324
Columns: 9


In [5]:
# Column names
print(df_data.columns.tolist())

['Unnamed: 0.1', 'Unnamed: 0', 'Organisation', 'Location', 'Date', 'Detail', 'Rocket_Status', 'Price', 'Mission_Status']


In [6]:
# First few rows
df_data.head()

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [6]:
# Data types and info
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0.1    4324 non-null   int64 
 1   Unnamed: 0      4324 non-null   int64 
 2   Organisation    4324 non-null   object
 3   Location        4324 non-null   object
 4   Date            4324 non-null   object
 5   Detail          4324 non-null   object
 6   Rocket_Status   4324 non-null   object
 7   Price           964 non-null    object
 8   Mission_Status  4324 non-null   object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data.

In [7]:
# Check for missing values
print("Missing values per column:")
print(df_data.isna().sum())
print(f"\nTotal missing values: {df_data.isna().sum().sum()}")

Missing values per column:
Unnamed: 0.1         0
Unnamed: 0           0
Organisation         0
Location             0
Date                 0
Detail               0
Rocket_Status        0
Price             3360
Mission_Status       0
dtype: int64

Total missing values: 3360


In [8]:
# Check for duplicates
print(f"Number of duplicate rows: {df_data.duplicated().sum()}")

Number of duplicate rows: 0


In [9]:
# Remove unnecessary columns (index columns)
df_data = df_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
print("Dropped unnecessary index columns")
df_data.head()

Dropped unnecessary index columns


,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


### Convert Date Column to Datetime

In [12]:
# Convert Date to datetime
df_data.loc[:, "Date"] = pd.to_datetime(df_data["Date"], format="%a %b %d, %Y %H:%M UTC", errors="coerce")

# Extract year
df_data.loc[:, "Year"] = df_data["Date"].dt.year

print(f"Date conversion complete. Years range from {df_data['Year'].min()} to {df_data['Year'].max()}")
df_data.head()

Date conversion complete. Years range from 1957.0 to 2020.0


,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Year
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,"2,020.00"
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,"2,020.00"
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00,Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,"2,020.00"
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,"2,020.00"
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00,Atlas V 541 | Perseverance,StatusActive,145.0,Success,"2,020.00"


## Descriptive Statistics

In [13]:
# Summary statistics
df_data.describe()

,Date,Year
count,4198,"4,198.00"
mean,1987-12-23 12:59:23.639828416,"1,987.46"
min,1957-10-04 19:28:00,"1,957.00"
25%,1972-05-05 17:21:00,"1,972.00"
50%,1985-01-24 06:21:30,"1,985.00"
75%,2002-10-05 00:25:30,"2,002.00"
max,2020-08-07 05:12:00,"2,020.00"
std,NaN,18.09


In [14]:
# Unique values in key columns
print(f"Unique Organizations: {df_data['Organisation'].nunique()}")
print(f"Unique Locations: {df_data['Location'].nunique()}")
print(f"Mission Status values: {df_data['Mission_Status'].unique()}")
print(f"Rocket Status values: {df_data['Rocket_Status'].unique()}")

Unique Organizations: 56
Unique Locations: 137
Mission Status values: ['Success' 'Failure' 'Prelaunch Failure' 'Partial Failure']
Rocket Status values: ['StatusActive' 'StatusRetired']


## Number of Launches per Company

In [15]:
launches_per_company = df_data['Organisation'].value_counts()
print("Top 20 organizations by number of launches:")
print(launches_per_company.head(20))

Top 20 organizations by number of launches:
Organisation
RVSN USSR           1777
Arianespace          279
CASC                 251
General Dynamics     251
NASA                 203
VKS RF               201
US Air Force         161
ULA                  140
Boeing               136
Martin Marietta      114
SpaceX               100
MHI                   84
Northrop              83
Lockheed              79
ISRO                  76
Roscosmos             55
ILS                   46
Sea Launch            36
ISAS                  30
Kosmotras             22
Name: count, dtype: int64


In [16]:
# Visualize top organizations
top_orgs = launches_per_company.head(10)
fig = px.bar(x=top_orgs.values, 
             y=top_orgs.index, 
             orientation='h',
             title='Top 10 Organizations by Number of Launches',
             labels={'x': 'Number of Launches', 'y': 'Organization'},
             color=top_orgs.values,
             color_continuous_scale='viridis')
fig.update_layout(showlegend=False, height=500)
fig.show()

## Number of Launches per Location

In [17]:
launches_per_location = df_data['Location'].value_counts()
print("Top 10 locations by number of launches:")
print(launches_per_location.head(10))

Top 10 locations by number of launches:
Location
Site 31/6, Baikonur Cosmodrome, Kazakhstan           235
Site 132/1, Plesetsk Cosmodrome, Russia              216
Site 43/4, Plesetsk Cosmodrome, Russia               202
Site 41/1, Plesetsk Cosmodrome, Russia               198
Site 1/5, Baikonur Cosmodrome, Kazakhstan            193
Site 132/2, Plesetsk Cosmodrome, Russia              174
Site 133/3, Plesetsk Cosmodrome, Russia              158
Site 43/3, Plesetsk Cosmodrome, Russia               138
LC-39A, Kennedy Space Center, Florida, USA           120
ELA-2, Guiana Space Centre, French Guiana, France    118
Name: count, dtype: int64


## Mission Status Analysis

In [18]:
mission_status = df_data['Mission_Status'].value_counts()
print(mission_status)

# Calculate failure rate
total_missions = len(df_data)
failed_missions = mission_status.get('Failure', 0) + mission_status.get('Partial Failure', 0)
failure_rate = (failed_missions / total_missions) * 100
print(f"\nOverall failure rate: {failure_rate:.2f}%")

Mission_Status
Success              3879
Failure               339
Partial Failure       102
Prelaunch Failure       4
Name: count, dtype: int64

Overall failure rate: 10.20%


In [19]:
# Pie chart of mission status
fig = px.pie(values=mission_status.values, 
             names=mission_status.index,
             title='Mission Status Distribution',
             color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Extract Country from Location

In [21]:
# Extract country from location string
def extract_country(location):
    if pd.isna(location):
        return "Unknown"
    location_parts = location.split(",")
    return location_parts[-1].strip()


df_data.loc[:, "Country"] = df_data["Location"].apply(extract_country)

# Show unique countries
print(f"Unique countries: {df_data['Country'].nunique()}")
print(f"\nTop 10 countries by launches:")
print(df_data["Country"].value_counts().head(10))

Unique countries: 22

Top 10 countries by launches:
Country
Russia           1395
USA              1344
Kazakhstan        701
France            303
China             268
Japan             126
India              76
Pacific Ocean      36
New Zealand        13
Iran               13
Name: count, dtype: int64


## How many space missions have there been?

Show the total number of launches over time.

In [23]:
# Convert Year to integer (remove decimals and format as whole numbers)
df_data.loc[:, "Year"] = df_data["Year"].astype("Int64")

# Total launches
print(f"Total number of space missions: {len(df_data)}")

# Launches per year
launches_per_year = df_data.groupby("Year").size().reset_index(name="Launches")
print(f"\nLaunches per year (sample):")
print(launches_per_year.head(10))

Total number of space missions: 4324

Launches per year (sample):
   Year  Launches
0  1957         3
1  1958        22
2  1959        20
3  1960        38
4  1961        52
5  1962        81
6  1963        38
7  1964        56
8  1965        86
9  1966        98


/var/folders/gx/js1jjyls69d93s456wgtzc0w0000gn/T/ipykernel_25466/3612114612.py:2: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020,
 ...
 1958, 1958, 1958, 1958, 1958, 1958, 1958, 1957, 1957, 1957]
Length: 4324, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [24]:
# Line chart of launches over time
fig = px.line(launches_per_year, 
              x='Year', 
              y='Launches',
              title='Number of Space Launches Over Time',
              markers=True)
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches',
                  hovermode='x unified')
fig.show()

In [25]:
# Area chart for better visualization
fig = px.area(launches_per_year, 
              x='Year', 
              y='Launches',
              title='Space Launches Over Time (Area Chart)',
              color_discrete_sequence=['#636EFA'])
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches')
fig.show()

## Create a Chart Showing the Total Number of Launches by Country

Create a Sunburst Chart showing the number of launches by Country, Organisation, and Mission Status.

In [26]:
# Prepare data for sunburst
sunburst_data = df_data.groupby(['Country', 'Organisation', 'Mission_Status']).size().reset_index(name='Count')

# Create sunburst chart
fig = px.sunburst(sunburst_data, 
                  path=['Country', 'Organisation', 'Mission_Status'], 
                  values='Count',
                  title='Space Launches by Country, Organization, and Mission Status',
                  color='Count',
                  color_continuous_scale='RdYlBu_r',
                  height=700)
fig.show()

## Chart the Number of Launches by the Top 10 Organisations Over Time

How has the dominance of launches changed over time between the different players?

In [29]:
# Get top 10 organizations
top_10_orgs = df_data['Organisation'].value_counts().head(10).index.tolist()

# Filter data for top 10
top_orgs_data = df_data[df_data['Organisation'].isin(top_10_orgs)]

# Group by year and organization
org_launches_yearly = top_orgs_data.groupby(['Year', 'Organisation']).size().reset_index(name='Launches')

print("Sample data:")
print(org_launches_yearly.head(15))

Sample data:
    Year      Organisation  Launches
0   1957         RVSN USSR         2
1   1958              NASA         2
2   1958         RVSN USSR         5
3   1958      US Air Force         2
4   1959  General Dynamics         1
5   1959              NASA         1
6   1959         RVSN USSR         4
7   1959      US Air Force        10
8   1960  General Dynamics         5
9   1960              NASA         4
10  1960         RVSN USSR         8
11  1960      US Air Force        21
12  1961  General Dynamics         8
13  1961              NASA        12
14  1961         RVSN USSR         9


In [30]:
# Line chart showing launches by top organizations over time
fig = px.line(org_launches_yearly, 
              x='Year', 
              y='Launches', 
              color='Organisation',
              title='Number of Launches by Top 10 Organizations Over Time',
              markers=True)
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches',
                  hovermode='x unified',
                  height=600)
fig.show()

In [31]:
# Stacked area chart
fig = px.area(org_launches_yearly, 
              x='Year', 
              y='Launches', 
              color='Organisation',
              title='Space Launches by Top 10 Organizations (Stacked Area)',
              line_group='Organisation')
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches',
                  height=600)
fig.show()

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991.

In [33]:
# Categorize countries into USA, USSR, or Other
def categorize_country(country):
    usa_keywords = ["USA", "United States", "New Mexico", "Pacific"]
    ussr_keywords = ["Russia", "Kazakhstan", "Soviet Union", "USSR"]

    if any(keyword in country for keyword in usa_keywords):
        return "USA"
    elif any(keyword in country for keyword in ussr_keywords):
        return "USSR"
    else:
        return "Other"


df_data.loc[:, "Superpower"] = df_data["Country"].apply(categorize_country)

# Filter for Cold War period (up to 1991)
cold_war_df = df_data[df_data["Year"] <= 1991].copy()

print(f"Total missions during Cold War: {len(cold_war_df)}")
print(f"\nLaunches by superpower during Cold War:")
print(cold_war_df["Superpower"].value_counts())

Total missions during Cold War: 2520

Launches by superpower during Cold War:
Superpower
USSR     1709
USA       644
Other     167
Name: count, dtype: int64


## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches.

In [34]:
# Count launches by superpower
cold_war_launches = cold_war_df['Superpower'].value_counts()

# Create pie chart
fig = px.pie(values=cold_war_launches.values, 
             names=cold_war_launches.index,
             title='Cold War Space Race: Total Launches by Superpower (1957-1991)',
             color_discrete_map={'USA': '#B22234', 'USSR': '#DA291C', 'Other': '#CCCCCC'})
fig.update_traces(textposition='inside', 
                  textinfo='percent+label+value',
                  textfont_size=14)
fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [35]:
# Get USA and USSR launches per year
usa_ussr_yearly = cold_war_df[cold_war_df['Superpower'].isin(['USA', 'USSR'])].groupby(['Year', 'Superpower']).size().reset_index(name='Launches')

print("Sample data:")
print(usa_ussr_yearly.head(10))

Sample data:
   Year Superpower  Launches
0  1957        USA         1
1  1957       USSR         2
2  1958        USA        17
3  1958       USSR         5
4  1959        USA        16
5  1959       USSR         4
6  1960        USA        30
7  1960       USSR         8
8  1961        USA        43
9  1961       USSR         9


In [37]:
# Line chart comparing USA and USSR
fig = px.line(usa_ussr_yearly, 
              x='Year', 
              y='Launches', 
              color='Superpower',
              title='USA vs USSR: Annual Space Launches During the Cold War',
              markers=True,
              color_discrete_map={'USA': "#2224B2", 'USSR': '#DA291C'})
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches',
                  hovermode='x unified',
                  legend_title_text='Country')
fig.show()

In [39]:
# Bar chart comparison
fig = px.bar(usa_ussr_yearly, 
             x='Year', 
             y='Launches', 
             color='Superpower',
             title='USA vs USSR: Annual Space Launches (Bar Chart)',
             barmode='group',
             color_discrete_map={'USA': '#2224B2', 'USSR': '#DA291C'})
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches',
                  legend_title_text='Country')
fig.show()

## Chart the Total Number of Mission Failures Year on Year.

In [40]:
# Get failures per year
failures_per_year = df_data[df_data['Mission_Status'].isin(['Failure', 'Partial Failure'])].groupby('Year').size().reset_index(name='Failures')

print("Mission failures per year (first 15 years):")
print(failures_per_year.head(15))

Mission failures per year (first 15 years):
    Year  Failures
0   1957         1
1   1958        16
2   1959        12
3   1960        19
4   1961        20
5   1962        16
6   1963         9
7   1964         9
8   1965        12
9   1966        16
10  1967        16
11  1968         9
12  1969        16
13  1970        14
14  1971        14


In [41]:
# Line chart of failures
fig = px.line(failures_per_year, 
              x='Year', 
              y='Failures',
              title='Total Mission Failures Per Year',
              markers=True,
              color_discrete_sequence=['#EF553B'])
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Failures')
fig.show()

In [43]:
# Compare failures by superpower during Cold War
cold_war_failures = cold_war_df[cold_war_df['Mission_Status'].isin(['Failure', 'Partial Failure'])]
failures_by_superpower = cold_war_failures.groupby(['Year', 'Superpower']).size().reset_index(name='Failures')
failures_usa_ussr = failures_by_superpower[failures_by_superpower['Superpower'].isin(['USA', 'USSR'])]

fig = px.line(failures_usa_ussr, 
              x='Year', 
              y='Failures', 
              color='Superpower',
              title='Mission Failures: USA vs USSR During Cold War',
              markers=True,
              color_discrete_map={'USA': '#2224B2', 'USSR': '#DA291C'})
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Failures',
                  hovermode='x unified')
fig.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time?

In [46]:
# Calculate failure percentage per year
yearly_stats = (
    df_data.groupby("Year")
    .agg(
        Total=("Mission_Status", "count"),
        Failures=(
            "Mission_Status",
            lambda x: ((x == "Failure") | (x == "Partial Failure")).sum(),
        ),
    )
    .reset_index()
)

yearly_stats.loc[:, "Failure_Rate"] = (
    yearly_stats["Failures"] / yearly_stats["Total"]
) * 100

print("Failure rate per year (first 15 years):")
print(yearly_stats.head(15))

Failure rate per year (first 15 years):
    Year  Total  Failures  Failure_Rate
0   1957      3         1         33.33
1   1958     22        16         72.73
2   1959     20        12         60.00
3   1960     38        19         50.00
4   1961     52        20         38.46
5   1962     81        16         19.75
6   1963     38         9         23.68
7   1964     56         9         16.07
8   1965     86        12         13.95
9   1966     98        16         16.33
10  1967    102        16         15.69
11  1968    100         9          9.00
12  1969    101        16         15.84
13  1970    106        14         13.21
14  1971    116        14         12.07


In [47]:
# Plot failure percentage over time
fig = px.line(yearly_stats, 
              x='Year', 
              y='Failure_Rate',
              title='Mission Failure Rate Over Time (%)',
              markers=True,
              color_discrete_sequence=['#AB63FA'])
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Failure Rate (%)',
                  yaxis_range=[0, yearly_stats['Failure_Rate'].max() + 5])
fig.add_hline(y=failure_rate, 
              line_dash="dash", 
              line_color="red",
              annotation_text=f"Overall Average: {failure_rate:.2f}%",
              annotation_position="top right")
fig.show()

In [50]:
# Failure rate by superpower during Cold War
cold_war_stats = (
    cold_war_df[cold_war_df["Superpower"].isin(["USA", "USSR"])]
    .groupby(["Year", "Superpower"])
    .agg(
        Total=("Mission_Status", "count"),
        Failures=(
            "Mission_Status",
            lambda x: ((x == "Failure") | (x == "Partial Failure")).sum(),
        ),
    )
    .reset_index()
)

cold_war_stats.loc[:, "Failure_Rate"] = (
    cold_war_stats["Failures"] / cold_war_stats["Total"]
) * 100

fig = px.line(
    cold_war_stats,
    x="Year",
    y="Failure_Rate",
    color="Superpower",
    title="Mission Failure Rate: USA vs USSR During Cold War",
    markers=True,
    color_discrete_map={"USA": "#3A22B2", "USSR": "#DA291C"},
)
fig.update_layout(
    xaxis_title="Year", yaxis_title="Failure Rate (%)", hovermode="x unified"
)
fig.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including 2020)

Do the results change if we only look at the number of successful launches?

In [52]:
# Cumulative launches by superpower
cumulative_data = (
    df_data[df_data["Superpower"].isin(["USA", "USSR"])]
    .groupby(["Year", "Superpower"])
    .size()
    .reset_index(name="Launches")
)
cumulative_data.loc[:, "Cumulative_Launches"] = cumulative_data.groupby("Superpower")[
    "Launches"
].cumsum()

print("Cumulative launches (first 15 years):")
print(cumulative_data.head(15))

Cumulative launches (first 15 years):
    Year Superpower  Launches  Cumulative_Launches
0   1957        USA         1                    1
1   1957       USSR         2                    2
2   1958        USA        17                   18
3   1958       USSR         5                    7
4   1959        USA        16                   34
5   1959       USSR         4                   11
6   1960        USA        30                   64
7   1960       USSR         8                   19
8   1961        USA        43                  107
9   1961       USSR         9                   28
10  1962        USA        59                  166
11  1962       USSR        22                   50
12  1963        USA        15                  181
13  1963       USSR        23                   73
14  1964        USA        23                  204


In [54]:
# Plot cumulative launches
fig = px.line(cumulative_data, 
              x='Year', 
              y='Cumulative_Launches', 
              color='Superpower',
              title='Cumulative Space Launches: USA vs USSR (All Time)',
              markers=True,
              color_discrete_map={'USA': "#4422B2", 'USSR': '#DA291C'})
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Cumulative Number of Launches',
                  hovermode='x unified',
                  height=600)
fig.show()

In [56]:
# Cumulative SUCCESSFUL launches only
successful_launches = df_data[
    (df_data["Superpower"].isin(["USA", "USSR"]))
    & (df_data["Mission_Status"] == "Success")
]
cumulative_success = (
    successful_launches.groupby(["Year", "Superpower"])
    .size()
    .reset_index(name="Launches")
)
cumulative_success.loc[:, "Cumulative_Successful_Launches"] = (
    cumulative_success.groupby("Superpower")["Launches"].cumsum()
)

fig = px.line(
    cumulative_success,
    x="Year",
    y="Cumulative_Successful_Launches",
    color="Superpower",
    title="Cumulative SUCCESSFUL Space Launches: USA vs USSR",
    markers=True,
    color_discrete_map={"USA": "#2E22B2", "USSR": "#DA291C"},
)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Cumulative Successful Launches",
    hovermode="x unified",
    height=600,
)
fig.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020?

In [57]:
# Get top organization per year
org_per_year = df_data.groupby(['Year', 'Organisation']).size().reset_index(name='Launches')
top_org_per_year = org_per_year.loc[org_per_year.groupby('Year')['Launches'].idxmax()]

print("Top organization by year (sample):")
print(top_org_per_year.head(15))
print("\n...")
print(top_org_per_year.tail(10))

Top organization by year (sample):
    Year  Organisation  Launches
0   1957     RVSN USSR         2
2   1958          AMBA         7
10  1959  US Air Force        10
15  1960  US Air Force        21
19  1961  US Air Force        23
23  1962  US Air Force        40
26  1963     RVSN USSR        23
30  1964     RVSN USSR        33
36  1965     RVSN USSR        52
42  1966     RVSN USSR        50
50  1967     RVSN USSR        73
56  1968     RVSN USSR        77
62  1969     RVSN USSR        82
74  1970     RVSN USSR        86
87  1971     RVSN USSR        90

...
     Year Organisation  Launches
505  2011          ULA        11
516  2012          ULA        10
530  2013          ULA        11
541  2014          ULA        14
543  2015  Arianespace        12
554  2016         CASC        22
580  2017       SpaceX        18
585  2018         CASC        37
601  2019         CASC        27
618  2020         CASC        19


In [58]:
# Create a bar chart
fig = px.bar(top_org_per_year, 
             x='Year', 
             y='Launches', 
             color='Organisation',
             title='Organization with Most Launches Each Year',
             hover_data=['Organisation'])
fig.update_layout(xaxis_title='Year', 
                  yaxis_title='Number of Launches by Leading Organization',
                  showlegend=True,
                  height=600)
fig.show()

In [59]:
# Focus on specific decades
print("Dominant organizations in the 1970s:")
print(top_org_per_year[(top_org_per_year['Year'] >= 1970) & (top_org_per_year['Year'] < 1980)]['Organisation'].value_counts())

print("\nDominant organizations in the 1980s:")
print(top_org_per_year[(top_org_per_year['Year'] >= 1980) & (top_org_per_year['Year'] < 1990)]['Organisation'].value_counts())

print("\nDominant organizations in 2018-2020:")
print(top_org_per_year[top_org_per_year['Year'].isin([2018, 2019, 2020])])

Dominant organizations in the 1970s:
Organisation
RVSN USSR    10
Name: count, dtype: int64

Dominant organizations in the 1980s:
Organisation
RVSN USSR    10
Name: count, dtype: int64

Dominant organizations in 2018-2020:
     Year Organisation  Launches
585  2018         CASC        37
601  2019         CASC        27
618  2020         CASC        19


# Summary of Key Findings

1. **Total Missions**: The dataset contains information on over 4,300 space missions from 1957 to 2020.

2. **Cold War Dominance**: During the Cold War (1957-1991), the USSR dominated with approximately 1,709 launches compared to the USA's 644 launches.

3. **Failure Rates**: Overall mission failure rate is around 10%, which has decreased significantly over time from over 70% in the late 1950s to under 5% in recent years.

4. **Modern Era**: In recent years (2018-2020), China's CASC (China Aerospace Science and Technology Corporation) has become the most active organization, with SpaceX also emerging as a major player.

5. **Launch Activity**: Space launch activity peaked during the Cold War era and has seen a resurgence in the 2010s with the growth of commercial spaceflight.

6. **Success Rate Improvement**: Both superpowers significantly improved their success rates over time, demonstrating advances in rocket technology and mission planning.